In [6]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from scraping import init_driver, to_excel

In [7]:
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")


In [8]:
# extract

driver.find_element(By.ID, "practiceButton").click()
time.sleep(2) 

data = driver.find_element(By.ID, "result-table").text
splitdata = data.split("\n")
columns = splitdata[0].split(" ")
values_split = [row.split(" ") for row in splitdata[1:]]

table = pd.DataFrame(values_split, columns=columns[:4])

#### Exploring strategies

In [9]:
def strategy(table, choices, chosenchoices):
    week = max(table['Week'].astype(int))
    if week > 1 and week - 1 < len(chosenchoices):
        prev_choice = chosenchoices[week-1]
        lift = (int(table['Sales'][week-1]) - int(table['Sales'][week-2])) / int(table['Sales'][week-2])
        lastchoice = max(chosenchoices)
        if lift < 0:
            print("Negative lift:", lift)
            return min(lastchoice+1, 3)
    return 0


In [12]:
attempts = 1000
all_data = []


for _ in range(attempts):
    chosenchoices = [0]
    driver.find_element(By.ID, "practiceButton").click()
    time.sleep(2) 

    for i in range(15):
        data = driver.find_element(By.ID, "result-table").text
        splitdata = data.split("\n")
        columns = splitdata[0].split(" ")
        values_split = [row.split(" ") for row in splitdata[1:]]

        table = pd.DataFrame(values_split, columns=columns[:4])
        choice = strategy(table, [0, 1, 2, 3], chosenchoices)
        chosenchoices.append(choice)

        buttons = [
            driver.find_element(By.ID, "maintainButton"),
            driver.find_element(By.ID, "tenButton"),
            driver.find_element(By.ID, "twentyButton"),
            driver.find_element(By.ID, "fortyButton")
        ]
        buttons[choice].click()
        time.sleep(2) 

        if choice == 3:
            data = driver.find_element(By.ID, "result-table").text
            splitdata = data.split("\n")
            columns = splitdata[0].split(" ")
            values_split = [row.split(" ") for row in splitdata[1:]]

            table = pd.DataFrame(values_split, columns=columns[:4])
            chosenchoices.extend([0] * (13 - i))
            break

    foresight = driver.find_element(By.CSS_SELECTOR, "div[style*='position:relative;top:10px;left:6px;']").text
    table['foresight'] = foresight
    table['Choice'] = chosenchoices
    table['diff'] = float(table['foresight'][0].split(": ")[3].replace("%", ""))
    table['revenue'] = float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$", "").replace(",", ""))
    table['perfect'] = float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$", "").replace(",", ""))
    
    all_data.append(table)


Negative lift: -0.037383177570093455
Negative lift: -0.1901840490797546
Negative lift: -0.37185929648241206
Negative lift: -0.2
Negative lift: -0.3263888888888889
Negative lift: -0.3145539906103286
Negative lift: -0.14814814814814814
Negative lift: -0.16470588235294117
Negative lift: -0.08461538461538462
Negative lift: -0.20161290322580644
Negative lift: -0.3652694610778443
Negative lift: -0.02631578947368421
Negative lift: -0.6016949152542372
Negative lift: -0.1827956989247312
Negative lift: -0.2980392156862745
Negative lift: -0.13793103448275862
Negative lift: -0.6538461538461539
Negative lift: -0.5588235294117647
Negative lift: -0.07317073170731707
Negative lift: -0.4791666666666667
Negative lift: -0.2081447963800905
Negative lift: -0.13114754098360656
Negative lift: -0.20754716981132076
Negative lift: -0.4530386740331492
Negative lift: -0.04716981132075472
Negative lift: -0.04568527918781726
Negative lift: -0.06382978723404255
Negative lift: -0.020202020202020204
Negative lift: -0.

IndexError: list index out of range

In [11]:
table

,Week,Price,Sales,Remaining
0,1,60,161,1839
1,2,60,148,1691
2,3,54,228,1463
3,4,54,247,1216
4,5,54,248,968
5,6,54,251,717
6,7,54,262,455
7,8,54,234,221
8,9,48,221,0
9,10,48,0,0
